In [26]:
import pandas as pd
from prophet import Prophet
import pickle
import requests

symbol=['AAPL', 'IBM', 'RELIANCE.BSE']

# Load the models and evaluate accuracy for each symbol
for i in symbol:
    model_path = f'models/stock_model_{i}.pkl'
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    # Load the data for the selected company using the Alpha Vantage API
    api_key = '0VSSNFD1KRGASNV9'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={i}&apikey={api_key}&outputsize=full'
    response = requests.get(url)
    data = response.json()

    # Convert the JSON response to a Pandas DataFrame
    time_series_data = data['Time Series (Daily)']
    timestamps = list(time_series_data.keys())
    close_prices = [float(data_point['4. close']) for data_point in time_series_data.values()]
    df = pd.DataFrame({
        'ds': pd.to_datetime(timestamps),  # Convert 'ds' column to datetime64[ns]
        'y': close_prices
    })

    # Evaluate accuracy of the model
    future_dates = model.make_future_dataframe(periods=len(df))
    forecast = model.predict(future_dates)
    forecast['ds'] = pd.to_datetime(forecast['ds'])  # Convert 'ds' column to datetime64[ns]
    forecast = forecast[['ds', 'yhat']]

    # Merge forecast with actual data for evaluation
    merged_df = pd.merge(df, forecast, on='ds', how='inner')

    # Calculate R-squared value
    y_actual = merged_df['y']
    y_predicted = merged_df['yhat']
    ss_res = ((y_actual - y_predicted) ** 2).sum()
    ss_tot = ((y_actual - y_actual.mean()) ** 2).sum()
    r2 = 1 - (ss_res / ss_tot)

    print(f"R-squared for {i}: {r2:.4f}")

R-squared for AAPL: 0.8890
R-squared for IBM: 0.9387
R-squared for RELIANCE.BSE: 0.9054
